In [90]:
import numpy as np
import pandas as pd
import yfinance as yf
import pymongo
import pprint

In [83]:
#read csv
NQ = pd.read_csv('nasdaq.csv')
NY = pd.read_csv('NYSE.csv')


In [45]:
#dataframe creation
l = pd.DataFrame(data = NQ)
r = pd.DataFrame(data = NY)
frames = [l,r]
df = pd.concat(frames)
df



,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$1.17,0.0234,2.041%,3.697035e+07,China,NaN,2323,Miscellaneous,Service to the Health Industry
1,AACI,Armada Acquisition Corp. I Common Stock,$9.86,-0.0300,-0.303%,2.041957e+08,United States,2021.0,170,Finance,Diversified Financial Service
2,AACIU,Armada Acquisition Corp. I Unit,$10.02,0.0000,0.00%,0.000000e+00,United States,2021.0,72,Finance,Diversified Financial Service
3,AACIW,Armada Acquisition Corp. I Warrant,$0.2798,-0.0002,-0.071%,0.000000e+00,United States,2021.0,10100,Finance,Diversified Financial Service
4,AADI,Aadi Bioscience Inc. Common Stock,$16.37,-0.0400,-0.244%,3.428201e+08,United States,NaN,5426,Health Care,Biotechnology: Pharmaceutical Preparations
...,...,...,...,...,...,...,...,...,...,...,...
3153,ZTS,Zoetis Inc. Class A Common Stock,$175.545,-1.3150,-0.744%,8.272567e+10,United States,2013.0,680314,Health Care,Biotechnology: Pharmaceutical Preparations
3154,ZUO,Zuora Inc. Class A Common Stock,$11.66,-0.8600,-6.869%,1.492480e+09,NaN,2018.0,139693,Technology,EDP Services
3155,ZVIA,Zevia PBC Class A Common Stock,$3.31,-0.1400,-4.058%,1.287267e+08,NaN,2021.0,30464,Consumer Non-Durables,Specialty Foods
3156,ZWS,Zurn Water Solutions Corporation Common Stock,$30.17,-0.9200,-2.959%,3.800440e+09,United States,2012.0,58480,Public Utilities,Environmental Services


In [98]:
filteredDF = df[(df['Market Cap'] > 10**12) | (df['Market Cap'] < 10000) & ]
filteredDF = filteredDF.Symbol
filteredDF = filteredDF.sort_values()
filteredDF

2       AACIU
3       AACIW
6        AAIN
9        AAPL
27      ACABU
        ...  
4867    ZNTEU
4868    ZNTEW
4873    ZTAQU
4874    ZTAQW
4879    ZWRKW
Name: Symbol, Length: 1241, dtype: object

In [102]:
#Creating DB
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['project']
NQc = db['NQc']

#Stock info
for i in filteredDF:
    NQ = yf.Ticker(i)
    NQc_id = NQc.insert_one(NQ.info).inserted_id


In [116]:
#Count the number of documents
NQc.count_documents({})

#Find and print the documents located in the state of California
print(NQc.find({'state':{'$in': ['CA']}}))

#Create a cursor using the find method in which the documents are sorted based on the key 'previousClose'.\
#Iterate over the cursor and print the stock symbol with the previousClose price.
for i in NQc.find({},{'exchange': 1,'previousClose' : 1}).sort('previousClose'):
    print(i)
#Create a pipeline to find the average value of the 'previousClose' price of stocks with a previousClose price\
#greater than $100.
db.NQc.aggregate([
               {'$match': {'previousClose', {'$gte':100}} },
               {'$group': { '$avg': '$previousClose'}}])
